In [1]:
import pandas as pd
import numpy as np
data_path = "./cb_db_csv/"

In [2]:
pd.__version__

'0.23.4'

# cb_objects

In [3]:
df_object = pd.read_csv(data_path + 'cb_objects.csv', encoding = "ISO-8859-1")
df_object = df_object.replace(r'\N', np.nan)
df_object.founded_at = pd.to_datetime(df_object.founded_at)
df_object=df_object[pd.notnull(df_object['founded_at'])]
len(df_object.index)

100441

## Helper Method

In [6]:
TermList = ['angel','seed','a', 'b','c','d','e','f','g']
            # 'convertible','crowd','crowd_equity','debt_round','grant',
            # 'partial','post_ipo_debt','post_ipo_equity','private_equity','secondary_market','unattributed']
def FundMultiHotEncoding(row):
    global TermList 
    termEncoding = np.zeros(len(TermList))
    if not pd.isna(row): 
        if (row in TermList) == True:
            index = TermList.index(row)
            termEncoding[index] = 1
            return termEncoding
        else:
            row = np.nan
    return row

## IPOs

In [245]:
df_ipo = pd.read_csv(data_path + 'cb_ipos.csv', encoding = "ISO-8859-1")
df_ipo = df_ipo.drop(['source_url','created_at','updated_at'],axis=1)
df_ipo = df_ipo.replace(r'\N', np.nan)
## drop object_id = null
## drop companies without id record
df_ipo = df_ipo[df_ipo.object_id.notnull()]
null_columns=df_ipo.columns[df_ipo.isnull().any()]
print(df_ipo[null_columns].isnull().sum())

## make timestamp 
df_ipo.public_at = pd.to_datetime(df_ipo.public_at)
df_ipo['time_since_founded'] = pd.Timedelta('nan')
print(len(df_ipo.index))

valuation_amount           1146
valuation_currency_code       2
raised_amount              1115
raised_currency_code        558
public_at                   595
dtype: int64
1254


- Before: 1254
- After: 426

In [ ]:
def getIPODuration(row):
    df_row = df_object.loc[df_object['id'] == row.object_id]
    if not df_row.empty:
        if pd.notnull(df_row['founded_at'].values) and pd.notnull(row['public_at']):
            duration = row.public_at - df_row.founded_at
            row['time_since_founded'] = duration.astype('timedelta64[D]').values
    return row
df_ipo = df_ipo.apply(getIPODuration, axis=1)
df_ipo = df_ipo[df_ipo['time_since_founded'].notnull()]
df_ipo = df_ipo.set_index(['object_id']).sort_index()
df_ipo = df_ipo.drop(['id','ipo_id','valuation_amount','valuation_currency_code','raised_amount','raised_currency_code',
                    'public_at','stock_symbol'],axis=1)


In [209]:
df_ipo.to_pickle('./dataframe_cleaned/df_ipo') 

## Acquisition
### ID list and # of acquistions

In [ ]:
df_acq = pd.read_csv(data_path + 'cb_acquisitions.csv', encoding = "ISO-8859-1")
df_acq = df_acq.drop(['source_url','created_at','updated_at'],axis=1)

df_acq = df_acq.replace(r'\N', np.nan)
df_acq = df_acq[df_acq['acquired_object_id'].notnull()]
df_acq = df_acq[df_acq['acquired_at'].notnull()]
null_columns=df_acq.columns[df_acq.isnull().any()]
print(df_acq[null_columns].isnull().sum())

#df_acq['price_currency_code'] = df_acq['price_currency_code'].apply(MultiHotEncoding)
#df_acq['term_code'] = df_acq['term_code'].apply(AcqMultiHotEncoding)
df_acq = pd.concat([df_acq, pd.get_dummies(df_acq['price_currency_code'])], axis=1, sort=False)
df_acq = pd.concat([df_acq, pd.get_dummies(df_acq['term_code'])], axis=1, sort=False)
df_acq

In [211]:
## Acquiring
df_acq.acquired_at = pd.to_datetime(df_acq.acquired_at)
df_acq['acquiring_time_since_founded'] = pd.Timedelta('nan')
def getAcquiringDuration(row):
    df_row = df_object.loc[df_object['id'] == row['acquiring_object_id']]
    if not df_row.empty:
        if pd.notnull(df_row['founded_at'].values) and pd.notnull(row['acquired_at']):
            duration = row['acquired_at'] - df_row['founded_at']
            row['acquiring_time_since_founded'] = duration.astype('timedelta64[D]').values
    return row
df_acq = df_acq.apply(getAcquiringDuration, axis=1)


In [212]:
## Acquired
df_acq['acquired_time_since_founded'] = pd.Timedelta('nan')
def getAcquiredDuration(row):
    df_row = df_object.loc[df_object['id'] == row['acquired_object_id']]
    if not df_row.empty:
        if pd.notnull(df_row['founded_at'].values) and pd.notnull(row['acquired_at']):
            duration = row['acquired_at'] - df_row['founded_at']
            row['acquired_time_since_founded'] = duration.astype('timedelta64[D]').values
    return row
df_acq = df_acq.apply(getAcquiredDuration, axis=1)


In [213]:
#print(len())
#df_acq = df_acq[df_acq['acquiring_time_since_founded'].notnull()]
#df_acq = df_acq[df_acq['acquired_time_since_founded'].notnull()]
# save df
df_acq.to_pickle('./dataframe_cleaned/df_acq') 
df_acq

,id,acquisition_id,acquiring_object_id,acquired_object_id,term_code,price_amount,price_currency_code,acquired_at,AUD,CAD,EUR,GBP,JPY,SEK,USD,cash,cash_and_stock,stock,acquiring_time_since_founded,acquired_time_since_founded
0,1,1,c:11,c:10,NaN,20000000,USD,2007-05-30,0,0,0,0,0,0,1,0,0,0,[10225.0],NaT
1,2,7,c:59,c:72,cash,60000000,USD,2007-07-01,0,0,0,0,0,0,1,1,0,0,[3219.0],[456.0]
2,3,8,c:24,c:132,cash,280000000,USD,2007-05-01,0,0,0,0,0,0,1,1,0,0,[7790.0],[1946.0]
3,4,9,c:59,c:155,cash,100000000,USD,2007-06-01,0,0,0,0,0,0,1,1,0,0,[3189.0],[1216.0]
4,5,10,c:212,c:215,cash,25000000,USD,2007-07-01,0,0,0,0,0,0,1,1,0,0,[4929.0],[2860.0]
5,6,11,c:227,c:228,NaN,7140000,USD,2003-08-05,0,0,0,0,0,0,1,0,0,0,[7078.0],[1496.0]
6,7,12,c:5,c:230,cash,NaN,USD,2007-07-01,0,0,0,0,0,0,1,1,0,0,[1246.0],NaT
7,8,14,c:318,c:314,cash,NaN,USD,2006-10-01,0,0,0,0,0,0,1,1,0,0,[-273.0],[638.0]
8,9,15,c:20,c:333,stock,1500000000,USD,2002-07-08,0,0,0,0,0,0,1,0,0,1,[2502.0],[1315.0]
9,10,16,c:342,c:341,cash,350000000,USD,2007-08-01,0,0,0,0,0,0,1,1,0,0,[30605.0],[669.0]


In [ ]:
## Acquiring

df_acquiring = df_acq.set_index(['acquiring_object_id','acquired_object_id']).sort_index()
df_acquiring.sort_values(by=['acquiring_time_since_founded'],inplace=True)
df_acquiring.drop(['id','acquisition_id','acquired_at','acquired_time_since_founded'],axis=1,inplace=True)
df_acquiring = df_acquiring[df_acquiring['acquiring_time_since_founded'].notnull()]
df_acquiring = df_acquiring[df_acquiring['acquiring_time_since_founded'].gt(0)]


In [222]:
df_acquiring = df_acquiring[df_acquiring['price_amount'].notnull()]

df_acquiring = df_acquiring.drop(['price_currency_code','term_code'],axis=1)

In [223]:
df_acquiring.to_pickle('./dataframe_cleaned/df_acquiring') 

In [224]:
df_acquiring

,,price_amount,AUD,CAD,EUR,GBP,JPY,SEK,USD,cash,cash_and_stock,stock,acquiring_time_since_founded
acquiring_object_id,acquired_object_id,,,,,,,,,,,,
c:60326,c:146208,1000000,0,0,0,0,0,0,1,0,0,1,[14.0]
c:15461,c:15460,70000,0,0,0,0,0,0,1,1,0,0,[25.0]
c:192899,c:193355,330000,0,0,0,0,0,0,1,0,1,0,[27.0]
c:138299,c:141275,15000,0,0,0,0,0,0,1,1,0,0,[29.0]
c:5457,c:5456,5000,0,0,0,0,0,0,1,1,0,0,[41.0]
c:195859,c:195858,100000,0,0,0,0,0,0,1,1,0,0,[42.0]
c:221698,c:190514,5000,0,0,0,0,0,0,1,1,0,0,[45.0]
c:171432,c:44839,3000,0,0,0,0,0,0,1,0,1,0,[49.0]
c:9212,c:9211,50000,0,0,1,0,0,0,0,1,0,0,[112.0]


Before kicking away the negative durations 
- 9532 rows × 4 columns 

After kicking away the negative durations 
- 6866 rows × 4 columns

After dropping entries with missing values price amount
- 1875 rows × 4 columns

## Acquired

Before kicking away the negative durations 
- 9532 rows × 4 columns

After kicking away the negative durations 
- 4821 rows × 4 columns


In [225]:
df_acquired = df_acq.set_index(['acquired_object_id']).sort_index()
df_acquired.sort_values(by=['acquired_time_since_founded'],inplace=True)
df_acquired.drop(['id','acquisition_id','acquiring_object_id','acquired_at','acquiring_time_since_founded','term_code','price_amount','price_currency_code'],axis=1,inplace=True)
df_acquired = df_acquired[df_acquired['acquired_time_since_founded'].gt(0)]

df_acquired.to_pickle('./dataframe_cleaned/df_acquired') 

In [226]:
df_acquired

,AUD,CAD,EUR,GBP,JPY,SEK,USD,cash,cash_and_stock,stock,acquired_time_since_founded
acquired_object_id,,,,,,,,,,,
c:22103,0,0,0,0,0,0,1,1,0,0,[18.0]
c:54651,0,0,0,0,0,0,1,0,0,0,[29.0]
c:2315,0,0,0,0,0,0,1,0,0,0,[31.0]
c:329,0,0,0,0,0,0,1,0,0,0,[31.0]
c:159552,0,0,0,0,0,0,1,0,0,1,[31.0]
c:54405,0,0,0,0,0,0,1,0,0,0,[59.0]
c:8565,0,0,0,0,0,0,1,0,0,0,[67.0]
c:86,0,0,0,0,0,0,1,0,0,0,[67.0]
c:45891,0,0,0,0,0,0,1,1,0,0,[75.0]


## Funding
### Todo: Replace entry missing funded_at and raised_amount_usd with -1

Non dropping raied amount and funded at 
- 31939 companies

After
- 27985 companies

After funding round code
- 18499 companies

After funding round code
- 18499 companies

In [228]:
df_funding= pd.read_csv(data_path + 'cb_funding_rounds.csv', encoding = "ISO-8859-1")
df_funding = df_funding.drop(['source_url','created_by','created_at','updated_at'],axis=1)
df_funding = df_funding.replace(r'\N', np.nan)

df_funding = df_funding[pd.notnull(df_funding['raised_amount_usd'])]
df_funding = df_funding[pd.notnull(df_funding['funded_at'])]

null_columns = df_funding.columns[df_funding.isnull().any()]
print(df_funding[null_columns].isnull().sum())

27985
pre_money_valuation_usd     46777
pre_money_valuation         46777
pre_money_currency_code     22090
post_money_valuation_usd    45650
post_money_valuation        45649
post_money_currency_code    19299
dtype: int64


In [ ]:
#drop those funding rounds out of concern
df_funding = df_funding[df_funding['funding_round_code'].isin(TermList)]
df_funding = pd.concat([df_funding, pd.get_dummies(df_funding['funding_round_code'])], axis=1, sort=False)

In [243]:
## make timestamp 
df_funding.funded_at = pd.to_datetime(df_funding.funded_at)
df_funding['time_since_founded'] = pd.Timedelta('nan')

In [244]:
def getDuration(row):
    df_row = df_object.loc[df_object['id'] == row.object_id]
    if not df_row.empty:
        if pd.notnull(df_row['founded_at'].values) and pd.notnull(row['funded_at']):
            duration = row.funded_at - df_row.founded_at
            row['time_since_founded'] = duration.astype('timedelta64[D]').values

    return row

df_funding = df_funding.apply(getDuration,axis=1)

## Funding Data
- Before kicking away NAN and negative durations: 31939 companies
- After: 14691 companies 

In [ ]:
#only positive date
df_funding = df_funding[df_funding['time_since_founded'].gt(-0.1)]
df_funding['time_since_founded'] = df_funding['time_since_founded'].astype(int)

In [249]:
df_funding = df_funding.set_index(['object_id','time_since_founded'])
df_funding.drop(['id','funding_round_id','funded_at','raised_amount','raised_currency_code','pre_money_valuation_usd','pre_money_valuation',
                 'pre_money_currency_code','post_money_valuation_usd','post_money_valuation',
                 'post_money_currency_code'],axis=1,inplace=True)

In [250]:
df_funding

funding_round_type funding_round_code  \
object_id time_since_founded                                         
c:4       781                          series-b                  b   
c:5       213                             angel              angel   
          455                          series-a                  a   
          790                          series-b                  b   
c:7299    1065                         series-b                  b   
c:9       214                          series-a                  a   
          273                          series-b                  b   
c:13      1399                         series-a               seed   
c:14      1581                         series-a                  a   
c:15      0                               angel              angel   
c:16      516                          series-b                  b   
          516                          series-a                  a   
c:15      151                          series-a                  a   
c:22      151                          series-a                  a   
c:23      220                          series-a                  a   
c:27      226                          series-b                  b   
c:30      243                          series-a                  a   
c:31      1461                        series-c+                  c   
          823                          series-b                  b   
c:34      851                          series-a                  a   
          1127                         series-b                  b   
c:27      501                         series-c+                  c   
          1537                        series-c+                  d   
c:38      212                          series-a                  a   
c:40      821                          series-b                  b   
          1308                        series-c+                  c   
c:43      182                             angel               seed   
c:44      395                         series-c+                  c   
          91                           series-a                  a   
          212                          series-b                  b   
...                                         ...                ...   
c:285830  30                           series-a               seed   
c:179637  955                         series-c+                  c   
c:184680  311                          series-a                  a   
c:167539  1787                         series-a                  a   
          2412                         series-b                  b   
c:73717   1136                         series-a                  a   
c:36444   6249                         series-a                  a   
c:275974  1075                         series-a                  a   
c:151059  1044                            angel               seed   
c:285592  83                              angel               seed   
c:38811   3006                        series-c+                  c   
c:203158  350                             angel               seed   
c:171632  756                             angel               seed   
          638                             angel               seed   
          453                             angel               seed   
c:23227   1243                        series-c+                  c   
          882                          series-a                  a   
c:284350  344                             angel               seed   
c:236066  2171                         series-a                  a   
c:27385   1276                         series-b                  b   
c:38711   464                          series-b                  b   
c:181145  559                          series-b                  b   
c:161321  1257                         series-a                  a   
          1093                            angel               seed   
c:17321   2654                         

In [251]:
len(df_funding.groupby('object_id').groups)

14691

In [252]:
df_funding.to_pickle('./dataframe_cleaned/df_funding')